# Intro to NetworkX

Why I like NetworkX:  
  - Multiple models
  - Easy to use
  - Extremely flexible
  - Lots of built in algorithms (with citations!)
  - Integrated graphing

# Multiple models

The following basic graph types are provided as Python classes:

`Graph`  
This class implements an undirected graph. It ignores multiple edges between two nodes. It does allow self-loop edges between a node and itself.  
  
`DiGraph`  
Directed graphs, that is, graphs with directed edges. Operations common to directed graphs, (a subclass of Graph).

`MultiGraph`  
A flexible graph class that allows multiple undirected edges between pairs of nodes. The additional flexibility leads to some degradation in performance, though usually not significant.  
  
`MultiDiGraph`  
A directed version of a MultiGraph.


|         | Undirected           | Directed  |
| ------------- |:-------------:| -----:|
| **Single Edge**    | Graph | DiGraph |  
| **Multi Edge**      | MultiGraph      |  MultiDiGraph |  


<div style=\"margin-top:200px\"/>

In [ ]:
import networkx as nx

# Easy to use

###### Create an empty graph

In [ ]:
g = nx.Graph()

###### Add some nodes

In [ ]:
g.add_nodes_from(xrange(5))
g.nodes()

In [ ]:
g.edges()

###### Add some edges

In [ ]:
g = nx.Graph()  # Start with a blank graph.
g.add_edges_from(zip(xrange(10), xrange(1,11)))  # Adding edges automatically adds the nodes.

In [ ]:
g.edges()

In [ ]:
nx.shortest_path(g, 1, 4)

###### Most things are stored in dicts

In [ ]:
g.adj

In [ ]:
# Get nodes adjacent to node 1.
g[1]

###### But in many cases you should use the methods of the graph class rather than the underlying dicts

In [ ]:
g.neighbors(1)

###### From the docs:

>Fast direct access to the graph data structure is also possible using subscript notation.
>
>**Warning**  
>Do not change the returned dict–it is part of the graph data structure and direct manipulation may leave the graph in an inconsistent state.

https://networkx.readthedocs.io/en/stable/tutorial/tutorial.html#accessing-edges

<div style="margin:300px"></div>

# Extremely flexible

### Nodes

From the docs:  
>In NetworkX, nodes can be any hashable object e.g. a text string, an image, an XML object, another Graph, a customized node object, etc.

###### Nodes can be strings

In [ ]:
g = nx.Graph()
g.add_nodes_from('abcd')
g.nodes()

###### Nodes can be tuples

In [ ]:
from numpy.random import choice
g = nx.Graph()
g.add_nodes_from(zip(xrange(5), 'abcde', choice([True, False], 5)))
g.nodes()

###### Nodes can be sets

In [ ]:
g = nx.Graph()
g.add_nodes_from(map(frozenset, zip(xrange(5), 'abcde', choice([True, False], 5))))
g.nodes()

###### Nodes types can be different

In [ ]:
g = nx.Graph()
g.add_nodes_from(zip(xrange(5), 'abcde', choice([True, False], 5)))
g.add_nodes_from(map(frozenset, zip(xrange(5), 'abcde', choice([True, False], 5))))
g.nodes()

###### Nodes can be custom objects

In [ ]:
class MyNode(object):
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return self.name

In [ ]:
a = MyNode('boo')
a

In [ ]:
# Custom objects are hashable by default in Python
hash(a)

In [ ]:
b = MyNode('urns')
b

In [ ]:
hash(b)

In [ ]:
b.name = 'boo-urns'
b

In [ ]:
# The hash is unchanged even after changing the object.
hash(b)

In [ ]:
a==b

In [ ]:
g = nx.Graph()
g.add_nodes_from([a,b])
g.adj

In [ ]:
g.add_edge(a, b)
g.adj

###### Directed example

In [ ]:
g = nx.DiGraph()
g.add_edge(a, b)  # Nodes do not have to be added first.
g.adj

<div style="margin:300px"></div>

### Custom objects as nodes

In [ ]:
class MyNode2(object):
    def __init__(self, name, phone, state):
        self.name = name
        self.phone = phone
        self.state = state
        
    def __repr__(self):
        return self.name

In [ ]:
# Create instances of custom objects
a = MyNode2('Matt', 1234, 'MO')
b = MyNode2('Bob', 1987, 'IL')
c = MyNode2('Stan', 4567, 'TN')
# Create a new graph and add the nodes and edge.
g = nx.Graph()
g.add_node(a)   
g.add_node(b)
g.add_node(c)
g.add_edges_from([(a, b), (a, c)])  # Could have skipped the three lines above this since nodes get added automatically.

In [ ]:
g.adj

In [ ]:
# Get the names and states of all of Matt's neighbors.
for n in g.neighbors('Matt'):
    print(n.name, n.state)

In [ ]:
# Maybe subset notation will work.
g['Matt']

In [ ]:
# Get the names and states of all of Matt's neighbor's.
for n in g.neighbors(a):
    print(n.name, n.state)

<div style="margin:300px"></div>

### Node Attributes

In [ ]:
g = nx.Graph()
g.add_node('Matt', phone=1234, state='MO')
g.add_node('Bob', phone=1987, state='IL')
g.add_node('Stan', {'phone':4567, 'state':'TN'})  # Different way to supply attributes - supply as dict
g.add_edges_from([('Matt', 'Bob'), ('Matt', 'Stan')])

In [ ]:
g.nodes()

In [ ]:
g.nodes(data=True)

In [ ]:
g.node['Matt']

In [ ]:
# Get the names and states of all of Matt's neighbors.
for n in g.neighbors('Matt'):
    print(n, g.node[n]['state'])

###### Get attribute for all nodes

In [ ]:
nx.get_node_attributes(g, 'state')

### Edge attributes

In [ ]:
g = nx.DiGraph()
g.add_edge('Matt', 'Bob', name='Matt to Bob')
g.add_edge('Matt', 'Stan', {'name':'Matt to Stan', 'color':'green'})  # Not all nodes/edges have to have the same attributes.

###### Multiple ways to see edges and edge attributes

In [ ]:
g.adj

In [ ]:
g.edge

In [ ]:
g['Matt']

In [ ]:
g['Matt']['Bob']

In [ ]:
g.get_edge_data('Matt', 'Bob')

In [ ]:
g.get_edge_data('Bob', 'Matt') # Returns None because edge does not exist (directed graph)

###### List all edges related to a node

In [ ]:
g.edges('Matt')

###### Add edge attribute after edge is created

In [ ]:
g.add_edge('Matt', 'Aaron')
g.adj

In [ ]:
g['Matt']['Aaron']['distance'] = 10
g.adj

This is one of the few times that modifying the underlying dicts directly is allowed:
https://networkx.github.io/documentation/development/reference/generated/networkx.Graph.get_edge_data.html

### Instance methods vs NetworkX methods 
>Most of the NetworkX API is provided by functions which take a graph object as an argument. Methods of the graph object are limited to basic manipulation and reporting. This provides modularity of code and documentation. 

http://networkx.readthedocs.io/en/stable/reference/introduction.html#networkx-basics

<div style="margin:300px"></div>

# Lots of built in algorithms (with citations!)

https://networkx.readthedocs.io/en/stable/reference/algorithms.html

# Integrated graphing

In [ ]:
pandemic_g = nx.read_graphml('pandemic.graphml.txt')

# dicts from city names to numbers, and from numbers to city names.
city_names_to_num = {tup[1]['label']:tup[0] for tup in pandemic_g.nodes(data=True)}
city_num_to_names = {tup[0]:tup[1]['label'] for tup in pandemic_g.nodes(data=True)}

# Relabel nodes to city names.
pandemic_g = nx.relabel_nodes(pandemic_g, city_num_to_names)

In [ ]:
%matplotlib inline

In [ ]:
nx.draw(pandemic_g)

In [ ]:
import json
from networkx.readwrite import json_graph
import flask

g = pandemic_g.copy()
for n in g:
    g.node[n]['name'] = n
    g.node[n]['r'] = 10
    g.node[n]['x'] = .5 * (1050 + g.node[n]['x'])
    g.node[n]['y'] = .6 * (1100 - g.node[n]['y'])
    
d = json_graph.node_link_data(g) # node-link format to serialize
# write json
json.dump(d, open('force/force.json','w'))
# Serve the file over http to allow for cross origin requests
app = flask.Flask('__main__', static_folder="force")
@app.route('/<path:path>')
def static_proxy(path):
  return app.send_static_file(path)
print('\nGo to http://localhost:8000/force.html to see the example\n')
app.run(port=8000)

## Docs have great starting tutorial

http://networkx.readthedocs.io/en/stable/tutorial/tutorial.html

http://networkx.readthedocs.io/en/stable/reference/introduction.html#networkx-basics

# More examples with the Pandemic graph

In [ ]:
pandemic_g = nx.read_graphml('pandemic.graphml.txt')

# dicts from city names to numbers, and from numbers to city names.
city_names_to_num = {tup[1]['label']:tup[0] for tup in pandemic_g.nodes(data=True)}
city_num_to_names = {tup[0]:tup[1]['label'] for tup in pandemic_g.nodes(data=True)}

# Relabel nodes to city names.
pandemic_g = nx.relabel_nodes(pandemic_g, city_num_to_names)

###### Page Rank

In [ ]:
results = nx.pagerank(pandemic_g)  # All the code you need for NetworkX to compute algorithm.
sorted([(k, v) for k,v in results.iteritems()], key=lambda tup: tup[1], reverse=True) # Sorting code.

###### k-clique

In [ ]:
list(nx.find_cliques(pandemic_g))

###### Clustering

In [ ]:
results = nx.clustering(pandemic_g)  # All the code you need for NetworkX to compute algorithm.
sorted([(k, v) for k,v in results.iteritems()], key=lambda tup: tup[1], reverse=True) # Sorting code.

###### Shortest Paths

In [ ]:
nx.shortest_path(pandemic_g, 'Atlanta', 'Hong Kong')

In [ ]:
nx.shortest_path(pandemic_g, 'Lima', 'Essen')

###### All shortest paths

In [ ]:
list(nx.all_shortest_paths(pandemic_g, 'Lima', 'Essen'))

###### Single source shortest paths

In [ ]:
for k,v in nx.single_source_shortest_path(pandemic_g, 'Atlanta').items():
    print('Atlanta : ' + k)
    print(' -> '.join(v))
    print('')

###### Average shortest path length (good alternative for graph diameter in analysis)

In [ ]:
nx.average_shortest_path_length(pandemic_g)